## Convert bird observations registered in artsobeservasjoner.no into a ebird.org record format

In [23]:
# !pip install pandas
# !pip install pyproj
# !pip install openpyxl

In [24]:
import pandas as pd
# import geopandas as gpd
# from shapely.geometry import Point
import pyproj
# import geodatasets

ao_source = "minefunn_tom2019.xlsx"
ebird_template = "ebird_record_format_template.csv"

In [25]:
ao_df = pd.read_excel(ao_source, skiprows=2)
print(list(ao_df.columns))
ao_df[['Superlokalitet', 'Lokalitetsnavn','Østkoordinat', 'Nordkoordinat', 'Nøyaktighet', 'Originale koordinater',]].head(2)

['Id', 'Taksonsorteringsrekkefølge', 'Valideringsstatus', 'Rødlistekategori', 'Artsnavn', 'Vitenskapelig navn', 'Autor', 'Skjermet funn', 'Antall', 'Enhet', 'Alder', 'Kjønn', 'Aktivitet', 'Metode', 'Superlokalitet', 'Lokalitetsnavn', 'Østkoordinat', 'Nordkoordinat', 'Nøyaktighet', 'Originale koordinater', 'Fylke', 'Kommune', 'Fylke.1', 'IKKE I NORGE', 'Ekstern id', 'Startdato', 'Stattidspunkt', 'Sluttdato', 'Sluttidspunkt', 'Kommentar', 'Ikke gjenfunnet', 'Usikker artsbestemmelse', 'Uspontan', 'Natursystem', 'Natursystem beskrivelse', 'Livsmedium', 'Vitenskapelig livsmediumnavn', 'Art som livsmedium, beskrivelse', 'Livsmedium.1', 'Livsmediumbeskrivelse', 'Min. dybde', 'Maks. dybde', 'Høyde min', 'Høyde maks', 'Offentlig samling', 'Privat samling', 'Samlingsnummer', 'Samlingsbeskrivelse', 'Artsbestemt av', 'Bestemmelsesdato', 'Bekrefter', 'Bekreftelsesdato', 'Redigeringsansvarlig', 'Rapportør', 'Observatører', 'Prosjekt']


,Superlokalitet,Lokalitetsnavn,Østkoordinat,Nordkoordinat,Nøyaktighet,Originale koordinater
0,Runde,Goksøyr,325230,6924160,200,"Ø16359, N6954796 Sone 33 (±200m) UTM(WGS 84)"
1,Runde,"Lundeura, Runde",324316,6923443,0,"Ø15379, N6954165 Sone 33 (±0m) UTM(WGS 84)"


In [26]:
# Play with coordinates
easting = 16359
northing = 6954796
zone = 33
p = pyproj.Proj(proj='utm', zone=zone, datum='WGS84')
lon, lat = p(easting, northing, inverse=True)


# gdf = gpd.GeoDataFrame(data, crs='EPSG:4326')
# gdf.plot()

In [27]:
lon,lat

(5.616887295628967, 62.407573133496484)

In [28]:
orig = "Ø16359, N6954796 Sone 33 (±200m) UTM(WGS 84)"
orig.split()

['Ø16359,', 'N6954796', 'Sone', '33', '(±200m)', 'UTM(WGS', '84)']

In [29]:
ebird_headers = list(pd.read_csv(ebird_template, sep=";").columns)
# Add these headers to the ao list before we populate them
for ebird_header in ebird_headers:
    ao_df["ebird-"+ebird_header]= ""
ebird_headers

['Common Name',
 'Genus',
 'Species',
 'Number',
 'Species Comments',
 'Location Name',
 'Latitude',
 'Longitude',
 'Date',
 'Start Time',
 'State/Province',
 'Country Code',
 'Protocol',
 'Number of Observers',
 'Duration',
 'All observations reported?',
 'Effort Distance Miles',
 'Effort area acres',
 'Submission Comments']

In [30]:
# Tried tp use the 
def digitsonly(text:str):
    return int("".join([c for c in text if c.isdigit()]))
def latlong_from_orig(orig_text:str):
    splits = orig_text.split(" ")
    easting = digitsonly(splits[0])
    northing = digitsonly(splits[1])
    zone = digitsonly(splits[3])
    precision = digitsonly(splits[4])
    p = pyproj.Proj(proj='utm', zone=zone, datum='WGS84')
    lon, lat = p(easting, northing, inverse=True)
    return lat, lon

latlong_from_orig("Ø16359, N6954796 Sone 33 (±200m) UTM(WGS 84)")

(62.407573133496484, 5.616887295628967)

In [34]:
ao_df.head(5)

,Id,Taksonsorteringsrekkefølge,Valideringsstatus,Rødlistekategori,Artsnavn,Vitenskapelig navn,Autor,Skjermet funn,Antall,Enhet,...,ebird-Start Time,ebird-State/Province,ebird-Country Code,ebird-Protocol,ebird-Number of Observers,ebird-Duration,ebird-All observations reported?,ebird-Effort Distance Miles,ebird-Effort area acres,ebird-Submission Comments
0,28996960,70384791,Ikke validert (funnet er ikke kvalitetssikret),NaN,toppskarv,Gulosus aristotelis,"(Linnaeus, 1761)",Nei,3,NaN,...,,,,,,,,,,
1,28996936,70384791,Ikke validert (funnet er ikke kvalitetssikret),NaN,toppskarv,Gulosus aristotelis,"(Linnaeus, 1761)",Nei,60,NaN,...,,,,,,,,,,
2,28521548,70413656,Ikke validert (funnet er ikke kvalitetssikret),NaN,fossekall,Cinclus cinclus,"(Linnaeus, 1758)",Nei,1,NaN,...,,,,,,,,,,
3,28521511,70410787,Ikke validert (funnet er ikke kvalitetssikret),NaN,blåstrupe,Luscinia svecica,"(Linnaeus, 1758)",Nei,1,NaN,...,,,,,,,,,,
4,28467966,70403698,Ikke validert (funnet er ikke kvalitetssikret),NaN,løvsanger,Phylloscopus trochilus,"(Linnaeus, 1758)",Nei,1,NaN,...,,,,,,,,,,


In [38]:

c = 'Common Name'
ao_df["ebird-"+c] = ao_df["Vitenskapelig navn"]

c = 'Number'
ao_df["ebird-"+c] = ao_df["Antall"]

c = 'Species Comments'
ao_df["ebird-"+c] = ao_df["Aktivitet"]

c =  'Date'
#TODO


# UTF to lat long
for i, row in ao_df.copy().iterrows():
    try:
        ao_df.at[i,'ebird-Latitude'], ao_df.at[i,'ebird-Longitude'] = latlong_from_orig(row["Originale koordinater"])
    except:
        pass

ao_df[[c for c in ao_df.columns if c.startswith("ebird")]].head(10)


,ebird-Common Name,ebird-Genus,ebird-Species,ebird-Number,ebird-Species Comments,ebird-Location Name,ebird-Latitude,ebird-Longitude,ebird-Date,ebird-Start Time,ebird-State/Province,ebird-Country Code,ebird-Protocol,ebird-Number of Observers,ebird-Duration,ebird-All observations reported?,ebird-Effort Distance Miles,ebird-Effort area acres,ebird-Submission Comments
0,Gulosus aristotelis,,,3,,,62.407573,5.616887,,,,,,,,,,,
1,Gulosus aristotelis,,,60,,,62.400713,5.599945,,,,,,,,,,,
2,Cinclus cinclus,,,1,,,61.839438,9.732315,,,,,,,,,,,
3,Luscinia svecica,,,1,,,61.854239,9.763308,,,,,,,,,,,
4,Phylloscopus trochilus,,,1,Sang/spill i hekketid og passende hekkebiotop,,,,,,,,,,,,,,
5,Hirundo rustica,,,2,,,,,,,,,,,,,,,
6,Linaria cannabina,,,2,Par i passende hekkebiotop,,,,,,,,,,,,,,
7,Grus grus,,,2,Overflygende,,,,,,,,,,,,,,
8,Phalacrocorax carbo,,,1,,,60.580404,11.259222,,,,,,,,,,,
9,Phalacrocorax carbo,,,2,,,60.580404,11.259222,,,,,,,,,,,
